In [21]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from joblib import Parallel, delayed

In [ ]:
# Tải dữ liệu
file_path = '../Data/clean_data.csv'
df = pd.read_csv(file_path)

In [23]:
# Split the dataset into features (X) and the target variable (Y).
X = df.drop(columns = ['No', 'Price', 'Price_per_sqm', 'District', 'Month'])
y = df['Price']


numerical_features = ['Area', 'Bedrooms', 'WCs', 'Floors']

scaler = StandardScaler()

# Chuẩn hóa các đặc trưng số
X[numerical_features] = scaler.fit_transform(X[numerical_features])
X = X.values
y = y.values
# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [24]:

class Node:
    def __init__(self, feature = None, threshold = None, left = None, right = None, value = None):
        self.feature = feature      # chỉ số của đặc trưng để chia
        self.threshold = threshold  # ngưỡng để chia đặc trưng
        self.left = left            # cây con bên trái (nhỏ hơn hoặc bằng ngưỡng)
        self.right = right          # cây con bên phải (lớn hơn ngưỡng)
        self.value = value          # giá trị của nút lá (trung bình giá trị y)

In [25]:

class DecisionTreeRegressor:
    def __init__(self, max_depth = None, random_state = None):
        self.max_depth = max_depth
        self.random_state = random_state
        self.tree_ = None

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.tree_ = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._predict(inputs, self.tree_) for inputs in X])

    def _grow_tree(self, X, y, depth = 0):
        n_samples, n_features = X.shape
        if (self.max_depth is not None and depth >= self.max_depth) or len(set(y)) == 1:
            return Node(value=np.mean(y))

        feature_indices = np.random.choice(n_features, n_features, replace = False)
        best_feature, best_threshold = self._best_criteria(X, y, feature_indices)

        left_indices = X[:, best_feature] <= best_threshold
        right_indices = ~left_indices

        if not np.any(left_indices) or not np.any(right_indices):
            return Node(value = np.mean(y))

        left_tree = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right_tree = self._grow_tree(X[right_indices], y[right_indices], depth + 1)

        return Node(best_feature, best_threshold, left_tree, right_tree)

    def _best_criteria(self, X, y, feature_indices):
        best_gain = -1
        split_idx, split_threshold = None, None
        for feature_idx in feature_indices:
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                gain = self._information_gain(y, X[:, feature_idx], threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature_idx
                    split_threshold = threshold
        return split_idx, split_threshold

    def _information_gain(self, y, feature_values, threshold):
        parent_loss = self._mse(y)
        left_indices = feature_values <= threshold
        right_indices = ~left_indices
        if not np.any(left_indices) or not np.any(right_indices):
            return 0
        left_loss = self._mse(y[left_indices])
        right_loss = self._mse(y[right_indices])
        n = len(y)
        n_left = len(y[left_indices])
        n_right = len(y[right_indices])
        child_loss = (n_left / n) * left_loss + (n_right / n) * right_loss
        return parent_loss - child_loss

    def _mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)

    def _predict(self, inputs, node):
        while node.left:
            if inputs[node.feature] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value


In [26]:
class RandomForestRegressor:
    def __init__(self, n_estimators = 100, max_depth = None, max_features = 'sqrt', random_state = None, n_jobs = -1):
        """
        Khởi tạo mô hình RandomForestRegressor.

        Tham số:
        - n_estimators (int): Số lượng cây quyết định trong rừng.
        - max_depth (int): Độ sâu tối đa của mỗi cây quyết định. Nếu None, các cây sẽ được phát triển đầy đủ.
        - max_features (str): Số lượng đặc trưng tối đa được xem xét để phân chia tại mỗi nút.
        - random_state (int): Seed để đảm bảo tính nhất quán của random state.
        - n_jobs (int): Số lượng công việc song song hóa.
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state
        self.n_jobs = n_jobs
        self.estimators = []

    def _bootstrap_sample(self, X, y):
        """
        Tạo một mẫu bootstrap từ dữ liệu gốc.

        Tham số:
        - X (numpy array, shape = [n_samples, n_features]): Dữ liệu đầu vào.
        - y (numpy array, shape = [n_samples]): Nhãn của dữ liệu huấn luyện.

        Returns:
        - X_sample (numpy array, shape = [n_samples, n_features]): Mẫu bootstrap của dữ liệu đầu vào.
        - y_sample (numpy array, shape = [n_samples]): Nhãn của mẫu bootstrap.
        """
        n_samples = len(X)
        indices = np.random.choice(n_samples, size=n_samples, replace = True)
        return X[indices], y[indices]

    def fit(self, X, y):
        """
        Huấn luyện mô hình RandomForest dựa trên dữ liệu huấn luyện.

        Tham số:
        - X (numpy array, shape = [n_samples, n_features]): Dữ liệu đầu vào.
        - y (numpy array, shape = [n_samples]): Nhãn của dữ liệu huấn luyện.
        """
        np.random.seed(self.random_state)
        def _fit_single_tree(random_state):
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree = DecisionTreeRegressor(max_depth = self.max_depth, random_state = random_state)
            tree.fit(X_sample, y_sample)
            return tree

        random_states = np.random.randint(0, 10000, size = self.n_estimators)
        self.estimators = Parallel(n_jobs = self.n_jobs)(delayed(_fit_single_tree)(rs) for rs in random_states)

    def predict(self, X):
        """
        Dự đoán giá trị cho dữ liệu đầu vào mới.

        Tham số:
        - X (numpy array, shape = [n_samples, n_features]): Dữ liệu đầu vào cần dự đoán.

        Returns:
        - predictions (numpy array, shape = [n_samples]): Giá trị dự đoán tương ứng với mỗi mẫu đầu vào.
        """
        all_predictions = np.zeros((len(X), self.n_estimators))
        for i, tree in enumerate(self.estimators):
            all_predictions[:, i] = tree.predict(X)
        return np.mean(all_predictions, axis = 1)

In [27]:
# Train the Random Forest model
random_forest = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42)
random_forest.fit(X_train, y_train)

# Predict house prices on the test set
y_pred_forest = random_forest.predict(X_test)

# Evaluate model performance
print("Random Forest Regression - MSE:", mean_squared_error(y_test, y_pred_forest))  # #fix
print("Random Forest Regression - MAE:", mean_absolute_error(y_test, y_pred_forest))  # #fix
print("Random Forest Regression - R^2:", r2_score(y_test, y_pred_forest))  # #fix

Random Forest Regression - MSE: 34.09958081387315
Random Forest Regression - MAE: 2.46861951353079
Random Forest Regression - R^2: 0.7695719800110223


In [28]:
# Hàm dự đoán giá nhà dựa trên đầu vào của người dùng
def predict_house_price(area, bedrooms, wcs, floors, legal_status, district_encoded, is_frontage, is_alley):
    # Tạo DataFrame từ các tham số đầu vào
    input_data = pd.DataFrame({
        'Area': [area],
        'Bedrooms': [bedrooms],
        'WCs': [wcs],
        'Floors': [floors],
        'Legal_status': [legal_status],
        'District_encoded': [district_encoded],
        'Is_Frontage': [is_frontage],
        'Is_Alley': [is_alley]
    })
    # Chuẩn hóa dữ liệu đầu vào
    input_data[numerical_features] = scaler.transform(input_data[numerical_features])  # #fix
    input_data = input_data.values  # #fix
    # Dự đoán giá nhà
    prediction = random_forest.predict(input_data)
    return prediction[0]

In [29]:
# Ứng dụng GUI
class HousePriceApp:
    def __init__(self, root):
        self.root = root
        self.root.title("House Price Prediction")  # Đặt tiêu đề cho cửa sổ
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)  # Gọi hàm on_closing khi cửa sổ bị đóng

        # Tạo các trường nhập liệu
        self.area_label = ttk.Label(root, text="Area")
        self.area_label.grid(row=0, column=0, padx=10, pady=10)
        self.area_entry = ttk.Entry(root)
        self.area_entry.grid(row=0, column=1, padx=10, pady=10)

        self.bedrooms_label = ttk.Label(root, text="Bedrooms")
        self.bedrooms_label.grid(row=1, column=0, padx=10, pady=10)
        self.bedrooms_entry = ttk.Entry(root)
        self.bedrooms_entry.grid(row=1, column=1, padx=10, pady=10)

        self.wcs_label = ttk.Label(root, text="WCs")
        self.wcs_label.grid(row=2, column=0, padx=10, pady=10)
        self.wcs_entry = ttk.Entry(root)
        self.wcs_entry.grid(row=2, column=1, padx=10, pady=10)

        self.floors_label = ttk.Label(root, text="Floors")
        self.floors_label.grid(row=3, column=0, padx=10, pady=10)
        self.floors_entry = ttk.Entry(root)
        self.floors_entry.grid(row=3, column=1, padx=10, pady=10)

        self.legal_status_label = ttk.Label(root, text="Legal Status")
        self.legal_status_label.grid(row=4, column=0, padx=10, pady=10)
        self.legal_status_entry = ttk.Entry(root)
        self.legal_status_entry.grid(row=4, column=1, padx=10, pady=10)

        self.district_encoded_label = ttk.Label(root, text="District Encoded")
        self.district_encoded_label.grid(row=6, column=0, padx=10, pady=10)
        self.district_encoded_entry = ttk.Entry(root)
        self.district_encoded_entry.grid(row=6, column=1, padx=10, pady=10)

        self.frontage_1_label = ttk.Label(root, text="Is_Frontage")
        self.frontage_1_label.grid(row=8, column=0, padx=10, pady=10)
        self.frontage_1_entry = ttk.Entry(root)
        self.frontage_1_entry.grid(row=8, column=1, padx=10, pady=10)

        self.frontage_0_label = ttk.Label(root, text="Is_Alley")
        self.frontage_0_label.grid(row=9, column=0, padx=10, pady=10)
        self.frontage_0_entry = ttk.Entry(root)
        self.frontage_0_entry.grid(row=9, column=1, padx=10, pady=10)

        # Tạo nút dự đoán
        self.predict_button = ttk.Button(root, text="Predict Price", command=self.predict_price)
        self.predict_button.grid(row=11, column=0, columnspan=2, padx=10, pady=10)

        # Tạo nhãn để hiển thị kết quả
        self.result_label = ttk.Label(root, text="Predicted Price: ")
        self.result_label.grid(row=12, column=0, columnspan=2, padx=10, pady=10)

    def predict_price(self):
        try:
            # Lấy đầu vào từ người dùng
            area = float(self.area_entry.get())
            bedrooms = int(self.bedrooms_entry.get())
            wcs = int(self.wcs_entry.get())
            floors = int(self.floors_entry.get())
            legal_status = int(self.legal_status_entry.get())
            district_encoded = int(self.district_encoded_entry.get())
            is_frontage = int(self.frontage_1_entry.get())
            is_alley = int(self.frontage_0_entry.get())

            # Dự đoán giá
            predicted_price = predict_house_price(area, bedrooms, wcs, floors, legal_status, district_encoded, is_frontage, is_alley)
            self.result_label.config(text=f"Predicted Price: {predicted_price}")
        except ValueError:
            self.result_label.config(text="Please enter valid values.")  # Hiển thị thông báo lỗi nếu có lỗi

    def on_closing(self):
        self.root.destroy()  # Đóng cửa sổ
        self.root.quit()  # Thoát ứng dụng


In [30]:
# Hàm chính để chạy ứng dụng
if __name__ == "__main__":
    root = tk.Tk()
    app = HousePriceApp(root)
    root.mainloop()